## Import libaries

In [ ]:
!pip uninstall dea_ml -y

In [ ]:
!pip install -e dea_ml
# !pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools

In [ ]:
import os
import json
import xarray as xr
import geopandas as gpd
from odc.stats.tasks import TaskReader
from odc.stats.model import OutputProduct

## Enter the (old) X and Y Tile ID of the tile you want to run

In [ ]:
tile_x = '+0030'  #+0031,+0003 +0030,-0002
tile_y = '-0002'

### Open tiles geojson and find the new tile-id for the location entered 

In [ ]:
tiles=gpd.read_file('https://github.com/digitalearthafrica/crop-mask/blob/main/testing/eastern_cropmask/data/gm_s2_2019-2019--P1Y.geojson?raw=true')
t=[]
for index, row in tiles.iterrows():
    x,y = tiles.loc[index]['title'].split(",")
    if (x==tile_x) & (y==tile_y): 
        x,y = int(x)+181, int(y)+77
        new_title = str(x)+","+str(y)
        t.append(x)
        t.append(y)
        print(new_title)

### Extract the tasks object for that tile

In [ ]:
name, version = 'gm_s2_semiannual', '0-1-0'
op = OutputProduct(
            name='gm_s2_semiannual',
            version='0-1-0',
            short_name='gm_s2_semiannual',
            location=f"s3://dummy-bucket/{name}/{version}",
            properties={"odc:file_format": "GeoTIFF"},
            measurements=['red'],
        )

taskdb = TaskReader('s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db', product=op)
task = taskdb.load_task(('2019--P1Y', t[0], t[1]))

### Optionally export tile geojson to view

In [ ]:
with open('task_tile_check.geojson', 'w') as fh:
    json.dump(task.geobox.extent.to_crs('epsg:4326').json, fh, indent=2)

### Find the index of the tile we want to run

In [ ]:
tile_index_to_run = []
all_tiles = list(taskdb.all_tiles)
for i, index in zip(all_tiles, range(0, len(all_tiles))):
    if (i[1]==t[0]) & (i[2]==t[1]):
        tile_index_to_run.append(index)
        print(index)

### Run the crop-mask algo using odc-stats

In [ ]:
# os.system("cm-pred run "\
#           "s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db "\
#           "--config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/plugin_product.yaml "\
#           "--plugin-config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/ml_config.yaml "\
#           "--resolution=40 "\
#           "--threads=10 "\
#           "--memory-limit=90Gi "\
#           "--location=file:///home/jovyan/git/crop-mask/production/{product}/{version} "+str(tile_index_to_run[0])
#          )

### Or can run with cmd line, will see print statements

In [ ]:
%%time
!cm-pred run s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db --config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/plugin_product.yaml  --plugin-config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/ml_config.yaml  --resolution=40 --threads=14 --memory-limit=90Gi --location=file:///home/jovyan/git/crop-mask/production/{product}/{version} 3009

## Plot the results

In [ ]:
a= 'x'+str(t[0])
b='y0'+str(t[1])
prob=xr.open_rasterio('crop_mask_eastern/0-1-0/'+a+'/'+b+'/2019--P1Y/crop_mask_eastern_'+a+b+'_2019--P1Y_prob.tif')
print(prob)
prob.squeeze().plot.imshow(size=6);

In [ ]:
mask=xr.open_rasterio('crop_mask_eastern/0-1-0/'+a+'/'+b+'/2019--P1Y/crop_mask_eastern_'+a+b+'_2019--P1Y_mask.tif')#.squeeze().plot.imshow(size=6);
print(mask)
mask.squeeze().plot.imshow(size=6);

In [ ]:
mode=xr.open_rasterio('crop_mask_eastern/0-1-0/'+a+'/'+b+'/2019--P1Y/crop_mask_eastern_'+a+b+'_2019--P1Y_filtered.tif').squeeze()#.plot.imshow(size=6);
print(mode)
mode.squeeze().plot.imshow(size=6);

### Remove all files

In [ ]:
!rm -r -f crop_mask_eastern